In [2]:
# Python code
import random

# === GAME & SERIES LOGIC ===

def game_win_prob(team_rating, opp_rating, home_field_advantage=0):
    return (team_rating + home_field_advantage) / (team_rating + home_field_advantage + opp_rating)

def simulate_game(team_rating, opp_rating, is_home, hfa=0.05):
    prob = game_win_prob(team_rating, opp_rating, hfa if is_home else 0)
    return random.random() < prob

def get_home_games(round_name):
    return {
        "wildcard":       [True, True, True],
        "divisional":     [True, True, False, False, True],
        "lcs":            [True, True, False, False, False, True, True],
        "world_series":   [True, True, False, False, False, True, True],
    }[round_name]

def simulate_series(team_A, team_B, round_name):
    series_format = {
        "wildcard":     (2, 3),
        "divisional":   (3, 5),
        "lcs":          (4, 7),
        "world_series": (4, 7)
    }[round_name]

    wins_needed, max_games = series_format
    home_games = get_home_games(round_name)
    team_A_wins = 0
    team_B_wins = 0

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")

    for i in range(max_games):
        is_home_for_A = home_games[i]
        winner = simulate_game(team_A['rating'], team_B['rating'], is_home_for_A)

        if winner:
            team_A_wins += 1
            print(f"Game {i+1}: {team_A['name']} wins")
        else:
            team_B_wins += 1
            print(f"Game {i+1}: {team_B['name']} wins")

        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} wins the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} wins the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B

# === BRACKET SETUP ===

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # -- WILDCARD --
    ALWC1 = simulate_series(teams_AL[5], teams_AL[4], "wildcard")
    ALWC2 = simulate_series(teams_AL[6], teams_AL[3], "wildcard")
    NLWC1 = simulate_series(teams_NL[5], teams_NL[4], "wildcard")
    NLWC2 = simulate_series(teams_NL[6], teams_NL[3], "wildcard")

    # -- DIVISIONAL --
    ALDS1 = simulate_series(teams_AL[1], ALWC2, "divisional")
    ALDS2 = simulate_series(teams_AL[2], ALWC1, "divisional")
    NLDS1 = simulate_series(teams_NL[1], NLWC2, "divisional")
    NLDS2 = simulate_series(teams_NL[2], NLWC1, "divisional")

    # -- LCS --
    ALCS = simulate_series(ALDS1, ALDS2, "lcs")
    NLCS = simulate_series(NLDS1, NLDS2, "lcs")

    # -- WORLD SERIES --
    WS_winner = simulate_series(ALCS, NLCS, "world_series")

    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")

# === DEFINE RATINGS ===

def compute_rating(win_percentage, run_diff_per_game):
    run_diff_norm = (run_diff_per_game + 3) / 6
    rating = (0.7 * win_percentage) + (0.3 * run_diff_norm) 
    return rating


# === DEFINE TEAMS ===

# Format: name, rating (0–100), seed (1 = top seed)
teams_AL = [
    None,  # pad for 1-based index
    {"name": "Tigers", "rating": compute_rating(.621, 1.18), "seed": 1},
    {"name": "Astros", "rating": compute_rating(.605, 0.57), "seed": 2},
    {"name": "Yankees", "rating": compute_rating(.558, 1.20), "seed": 3},
    {"name": "Blue Jays", "rating": compute_rating(.558, 0.07), "seed": 4},
    {"name": "Rays", "rating": compute_rating(.552, 0.79), "seed": 5},
    {"name": "Mariners", "rating": compute_rating(.523, 0.17), "seed": 6}
]

teams_NL = [
    None,
    {"name": "Dodgers", "rating": compute_rating(.632, 1.10), "seed": 1},
    {"name": "Cubs", "rating": compute_rating(.593, 1.26), "seed": 2},
    {"name": "Phillies", "rating": compute_rating(.586, 0.60), "seed": 3},
    {"name": "Mets", "rating": compute_rating(.563, 0.44), "seed": 4},
    {"name": "Brewers", "rating": compute_rating(.558, 0.66), "seed": 5},
    {"name": "Padres", "rating": compute_rating(.535, 0.13), "seed": 6}
]

# === RUN THE SIMULATION ===
simulate_postseason(teams_AL, teams_NL)


⚾ SIMULATING MLB POSTSEASON BRACKET


🎯 WILDCARD SERIES: Rays (Seed 5) vs Blue Jays (Seed 4)
Game 1: Rays wins
Game 2: Rays wins
🏆 Rays wins the wildcard series 2-0

🎯 WILDCARD SERIES: Mariners (Seed 6) vs Yankees (Seed 3)
Game 1: Yankees wins
Game 2: Yankees wins
🏆 Yankees wins the wildcard series 2-0

🎯 WILDCARD SERIES: Brewers (Seed 5) vs Mets (Seed 4)
Game 1: Brewers wins
Game 2: Brewers wins
🏆 Brewers wins the wildcard series 2-0

🎯 WILDCARD SERIES: Padres (Seed 6) vs Phillies (Seed 3)
Game 1: Padres wins
Game 2: Padres wins
🏆 Padres wins the wildcard series 2-0

🎯 DIVISIONAL SERIES: Tigers (Seed 1) vs Yankees (Seed 3)
Game 1: Yankees wins
Game 2: Tigers wins
Game 3: Tigers wins
Game 4: Yankees wins
Game 5: Tigers wins
🏆 Tigers wins the divisional series 3-2

🎯 DIVISIONAL SERIES: Astros (Seed 2) vs Rays (Seed 5)
Game 1: Rays wins
Game 2: Astros wins
Game 3: Rays wins
Game 4: Astros wins
Game 5: Rays wins
🏆 Rays wins the divisional series 3-2

🎯 DIVISIONAL SERIES: Dodgers (Seed 1)